In [1]:
! apt-get install default-jre

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [2]:
!java -version

openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [3]:
!pip install h2o

     |████████████████████████████████| 128.3MB 79kB/s 
  Created wheel for h2o: filename=h2o-3.30.0.4-py2.py3-none-any.whl size=128397038 sha256=ad6c3513bf5573792984b8df233309edc30036e78962d10853c815eddca2eb98
  Stored in directory: /root/.cache/pip/wheels/ad/bd/cc/0a8c7dffc4772df0bbfae4c1c7a9d64525fcd3ada9694dd8c3
Successfully built h2o


In [0]:
import h2o
from h2o.automl import H2OAutoML 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprl3n4l56
  JVM stdout: /tmp/tmprl3n4l56/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprl3n4l56/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_unknownUser_3fz53z
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv('/content/sample_data/deaths.csv')
df

,Unnamed: 0,Country,Population,Area,Density,GDP,handwashing_facilities,hospital_beds_per_100k,Airports,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20
0,0,Afghanistan,38928341.0,652230.0,58.3257,74792.0,37.746,0.50,22.0,33.000000,65.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,40.0,42.0,43.0,47.0,50.0,57.0,58.0,60.0,64.0,68.0,72.0,85.0,90.0,95.0,104.0,106.0,109.0,115.0,120.0,122.0,127.0,132.0,136.0,153.0,168.0,169.0,173.0,178.0,187.0,193.0,205.0,216.0,218.0,219.0,220.0,227.0,235.0,246.0,249.0
1,1,Albania,2877800.0,28748.0,100.2128,38078.0,NaN,2.89,5.0,41.153300,20.168300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.0,27.0,27.0,27.0,27.0,28.0,28.0,30.0,30.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,32.0,32.0,33.0,33.0,33.0,33.0,33.0
2,2,Algeria,43851043.0,2381741.0,18.0763,638363.0,83.741,1.90,44.0,28.033900,1.659600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,392.0,402.0,407.0,415.0,419.0,425.0,432.0,437.0,444.0,450.0,453.0,459.0,463.0,465.0,470.0,476.0,483.0,488.0,494.0,502.0,507.0,515.0,522.0,529.0,536.0,542.0,548.0,555.0,561.0,568.0,575.0,582.0,592.0,600.0,609.0,617.0,623.0,630.0,638.0,646.0
3,3,Andorra,77265.0,468.0,164.8333,NaN,NaN,NaN,NaN,42.506300,1.521800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.0,37.0,37.0,40.0,40.0,40.0,40.0,41.0,42.0,42.0,43.0,44.0,45.0,45.0,46.0,46.0,47.0,47.0,48.0,48.0,48.0,48.0,49.0,49.0,49.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0
4,4,Angola,32866268.0,1246700.0,25.5276,199329.0,26.664,NaN,25.0,-11.202700,17.873900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,179,West Bank and Gaza,5101416.0,NaN,NaN,NaN,NaN,NaN,NaN,31.952200,35.233200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
184,180,Western Sahara,597330.0,266000.0,2.1897,NaN,NaN,NaN,3.0,24.215500,-12.885800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
185,181,Yemen,29825968.0,527968.0,55.2343,70414.0,49.542,0.70,11.0,15.552727,48.516388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2

In [0]:
df['handwashing_facilities'] = df['handwashing_facilities'].fillna(df['handwashing_facilities'].mean())
df['hospital_beds_per_100k'] = df['hospital_beds_per_100k'].fillna(df['hospital_beds_per_100k'].median())

In [9]:
new_df = df.dropna()
new_df

,Unnamed: 0,Country,Population,Area,Density,GDP,handwashing_facilities,hospital_beds_per_100k,Airports,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,...,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20
0,0,Afghanistan,38928341.0,652230.0,58.3257,74792.0,37.7460,0.5000,22.0,33.000000,65.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,40.0,42.0,43.0,47.0,50.0,57.0,58.0,60.0,64.0,68.0,72.0,85.0,90.0,95.0,104.0,106.0,109.0,115.0,120.0,122.0,127.0,132.0,136.0,153.0,168.0,169.0,173.0,178.0,187.0,193.0,205.0,216.0,218.0,219.0,220.0,227.0,235.0,246.0,249.0
1,1,Albania,2877800.0,28748.0,100.2128,38078.0,50.4595,2.8900,5.0,41.153300,20.168300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.0,27.0,27.0,27.0,27.0,28.0,28.0,30.0,30.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,32.0,32.0,33.0,33.0,33.0,33.0,33.0
2,2,Algeria,43851043.0,2381741.0,18.0763,638363.0,83.7410,1.9000,44.0,28.033900,1.659600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,392.0,402.0,407.0,415.0,419.0,425.0,432.0,437.0,444.0,450.0,453.0,459.0,463.0,465.0,470.0,476.0,483.0,488.0,494.0,502.0,507.0,515.0,522.0,529.0,536.0,542.0,548.0,555.0,561.0,568.0,575.0,582.0,592.0,600.0,609.0,617.0,623.0,630.0,638.0,646.0
4,4,Angola,32866268.0,1246700.0,25.5276,199329.0,26.6640,2.3585,25.0,-11.202700,17.873900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
5,5,Antigua and Barbuda,97928.0,442.0,219.7240,2506.0,50.4595,3.8000,2.0,17.060800,-61.796400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,176,Uzbekistan,33469199.0,447400.0,73.7186,310738.0,50.4595,4.0000,13.0,41.377500,64.585300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,7.0,7.0,8.0,8.0,8.0,8.0,8.0,9.0,9.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,11.0,12.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,14.0,14.0,14.0,14.0,14.0
182,178,Vietnam,97338583.0,331212.0,291.2398,1035051.0,85.8470,2.6000,25.0,16.000000,108.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,181,Yemen,29825968.0,527968.0,55.2343,70414.0,49.5420,0.7000,11.0,15.552727,48.516388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [0]:
data = df.T[11:]
data = data.astype('float64')

In [11]:
train = h2o.H2OFrame(np.array(data)[:, :100].T)
test  = h2o.H2OFrame(np.array(data)[:, 100:].T)

# Identify predictors and response
x = train.columns
y = "C130"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)

aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

lb = aml.leaderboard
lbdf = lb.as_data_frame()
lbdf.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |███████████████████████████████
17:43:25.590: Skipping training of model GBM_5_AutoML_20200603_173621 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200603_173621.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 100.0.


█████████████████████████| 100%


,model_id,mean_per_class_error,logloss,rmse,mse
0,GBM_1_AutoML_20200603_173621,0.930631,7.146899,0.874624,0.764966
1,GBM_3_AutoML_20200603_173621,0.945946,3.720931,0.913291,0.834101
2,GBM_4_AutoML_20200603_173621,0.945946,3.695181,0.910285,0.828619
3,GBM_2_AutoML_20200603_173621,0.954955,3.735907,0.918565,0.843761
4,DeepLearning_1_AutoML_20200603_173621,0.971471,5.920874,0.988765,0.977656
